<a href="https://colab.research.google.com/github/mcneela/Protein-LM-HF-Finetune/blob/master/Protein_LM_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FineTuning DistilProtBert

## Installation and Data Prep

In [1]:
!pip -q install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.7 MB/s eta 0:00:00


In [2]:
# upload GB1 dataset (select from local disk)
from google.colab import files
uploaded = files.upload()

Saving gb1.tsv to gb1.tsv


In [8]:
uploaded = files.upload()

Saving GB1.py to GB1.py


In [10]:
!mv GB1.py /content/GB1

In [6]:
!mkdir -p /content/GB1/train
!mkdir -p /content/GB1/test
!mv gb1.tsv /content/GB1
!wc -l /content/GB1/gb1.tsv

!head -482476 /content/GB1/gb1.tsv >> /content/GB1/train/train.tsv
!tail -53609 /content/GB1/gb1.tsv >> /content/GB1/test/test.tsv

536085 /content/GB1/gb1.tsv


In [11]:
import datasets
import numpy as np
data = datasets.load_dataset("GB1")
print(data['train'][0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset gb1 downloaded and prepared to /root/.cache/huggingface/datasets/gb1/full_dataset/1.0.0/ebd790e5cda8d93bf33be0a85856438f1336ca2a21f0e9197475baf5a14da116. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'variant': 'Q1A', 'seq': 'MAYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE', 'num_mutations': 1, 'labels': 0.41732627153396606}


In [18]:
import numpy as np

## Model Training

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("mcneela/protein-lm-gb1")
model = AutoModelForSequenceClassification.from_pretrained("mcneela/protein-lm-gb1", num_labels=1)

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/589 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/924M [00:00<?, ?B/s]

Some weights of the model checkpoint at yarongef/DistilProtBert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yarongef/DistilProtBe

In [13]:
def tokenize_function(example):
    return tokenizer(example["seq"], truncation=True)


tokenized_datasets = data.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/483 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/54 [00:00<?, ?ba/s]

In [ ]:
!huggingface-cli login

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    rmse = mean_squared_error(labels, logits, squared=False)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)

    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}

# Specifiy the arguments for the trainer  
training_args = TrainingArguments(
    output_dir ='./protein-lm-gb1',          
    num_train_epochs = 3,
    per_device_train_batch_size = 64,   
    per_device_eval_batch_size = 20,   
    weight_decay = 0.01,               
    learning_rate = 2e-5,
    logging_dir = './logs',  
    save_total_limit = 10,
    load_best_model_at_end = True,     
    metric_for_best_model = 'rmse',    
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    push_to_hub = True,
) 

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics_for_regression,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/mcneela/protein-lm-gb1 into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.44k/882M [00:00<?, ?B/s]

In [ ]:
trainer.train()
trainer.push_to_hub()